# Basic Functionality

ghoul version 0.1.0

## Collapsing symbols

The purpose of ghoul is to randomly generate internally-consistent python objects.

The basic unit in ghoul is the `Symbol`. A symbol is an object in a "superposition": it contains many possible states, until it "collapses" to a concrete value.

In [1]:
from ghoul import Symbol 

In [2]:
# create a new symbol
number = Symbol([1, 2, 3])

In [3]:
# check the possible values of number
number

[1, 2, 3]

In [4]:
# force number to adopt a concrete value
number.Observe()

2

In [5]:
# after this, number no long contains other possibilities
number

2

Upon being observed, `number` collapsed to a concrete value of 2, but it could have taken any of its potential values.

## Symbolic objects

Symbols can represent superpositions of generic python objects.

In [6]:
class Fruit(object):
    pass

class Apple(Fruit):
    def __init__(self):
        self.name = 'apple'
        self.color = Symbol(['green', 'red'])
        
class Pear(Fruit):
    def __init__(self):
        self.name = 'pear'
        self.color = Symbol(['green', 'yellow'])

In [7]:
# define a superposed instance of Fruit
fruit = Symbol(Fruit)

In [8]:
# check the possible attributes of the fruit
print(fruit)
print(fruit.name)
print(fruit.color)

[<__main__.Apple object at 0x10b3eee10>, <__main__.Pear object at 0x10b3eeda0>]
['apple', 'pear']
['green', 'red', 'green', 'yellow']


`fruit` is in a superposition of Apple and Pear, and this is reflected in the superposed values of its attributes `name` and `color`.

## Top-down collapsing

In [9]:
# collapse the fruit
fruit.Observe()

In [10]:
# inspect its new values
print(fruit)
print(fruit.name)
print(fruit.color)

pear
['green', 'yellow']


After collapsing `fruit` to an instance of `Pear`, we know its name must be `'pear'`, but its color is still undefined, being either green or yellow. It is no longer possible for `fruit` to be red.

## Bottom-up collapsing

Critically, when any attribute of the object is observed and takes on a concrete value, all of its superposed attributes will change as necessary to maintain consistency with the observation.

### The consistency requirement:

**The state of a symbolic object will always be consistent with past observations.**

In [11]:
# create a new superposed instance of Fruit
fruit = Symbol(Fruit)

# observe the fruit's name
fruit.name.Observe()

apple

after observing `fruit.name`, it collapsed to `'apple'`. `fruit` must now adjust its possible values to be consistent with this name. 

`fruit` had two possible values, an `Apple` object and a `Pear` object. If `Pear` remains a possibility, then it would be possible for `fruit` to collapse to a `Pear` named `apple`.

This is inconsistent with the definition of the `Pear` object, so once `fruit.name` collapses to `'apple'`, `Pear` is removed as a possiblity in order to preserve internal consistency.

In [12]:
# check all of the attribute values now
print(fruit)
print(fruit.name)
print(fruit.color)

apple
['green', 'red']


Knowing that `fruit` is an `Apple` object still does not tell us everything about its color, because an `Apple` may be still be either green or red.



## Minimal restriction

Both `Apple` and `Pear` are consistent with the fruit having a green color. Let's see what happens if we force `fruit` to be green:

In [13]:
# create a new superposed instance of Fruit
fruit = Symbol(Fruit)

# force fruit to be green
fruit.color.Collapse('green')

['green', 'green']

In [14]:
# check all of the attribute values now
print(fruit)
print(fruit.name)
print(fruit.color)

[<__main__.Apple object at 0x10b404828>, <__main__.Pear object at 0x10b4048d0>]
['apple', 'pear']
['green', 'green']


The fruit being green is not sufficient to discriminate between an Apple or a Pear. Therefore `fruit` may be either an `Apple` or `Pear` object, and `fruit.name` can be either `'apple'` or `'pear'`.

## Subclass restriction

Let's define a few subclasses of `Apple`:

In [15]:
class GrannySmith(Apple):
    def __init__(self):
        super().__init__()
        self.color = 'green'
        
class Honeycrisp(Apple):
    def __init__(self):
        super().__init__()
        self.color = 'red'

Now when we define fruit as symbolic instance of `Fruit`, it takes on potential values of `GrannySmith`, `Honeycrisp`, or `Pear`:

In [16]:
# create a new superposed instance of Fruit
fruit = Symbol(Fruit)

print(fruit)
print(fruit.name)
print(fruit.color)

[<__main__.GrannySmith object at 0x10b3ee978>, <__main__.Honeycrisp object at 0x10b3ee7b8>, <__main__.Pear object at 0x10b3ee278>]
['apple', 'apple', 'pear']
['green', 'red', 'green', 'yellow']


We force `fruit` to collapse to `Apple`. It therefore restricts its potential values to only those which are instances of `Apple` or subclasses of `Apple`:

In [17]:
fruit.Collapse(Apple)
print(fruit)
print(fruit.name)
print(fruit.color)

[<__main__.GrannySmith object at 0x10b3ee978>, <__main__.Honeycrisp object at 0x10b3ee7b8>]
['apple', 'apple']
['green', 'red']
